<a href="https://colab.research.google.com/github/Adil1979/P6_Tag_Recommendation_Stackoverflow/blob/master/01_P6_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet6: Catégorisation automatique de questions**

## **1.Librairies**

In [46]:
!pip install feature_engine

In [47]:
!pip install plotly

In [48]:
!pip install chart-studio

In [49]:
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib as mpl
import chart_studio.plotly.plotly as py
from collections import Counter
#import plotly.plotly as py
import seaborn as sns
import datetime as dt
import calendar as cld
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from os import listdir
import glob
import missingno as msno
#from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.decomposition import PCA
from matplotlib import dates
import nltk
import re
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import string
from collections import Counter
from bs4 import BeautifulSoup
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Word2Vec #Word Embedding

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


SELECT Id, Body, Title, Tags

FROM Posts

WHERE Score >= 3 AND PostTypeId = 1

ORDER BY RAND()
 
ASC OFFSET 0 ROWS FETCH NEXT 50000 ROWS ONLY; de 0 à 50000

## **2.Data**

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [52]:
#df_analysis_01 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_01.csv')
#df_analysis_02 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_02.csv')
#df_analysis_03 = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/QueryResults_03.csv')
#frames = [df_analysis_01, df_analysis_02, df_analysis_03]
#df_analysis = pd.concat(frames)
#df_sample = df_analysis.sample(n=40000)
#df_sample.to_csv('df_sample.csv')

In [53]:
df_sample = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/df_sample.csv')
df_sample = df_sample.drop(['Unnamed: 0'], axis=1)
df_sample.head()

,Id,Body,Title,Tags
0,57694656,<p>Pandas does not seem to find all elements i...,Pandas not finding elements in Columns,<python><pandas>
1,10475856,<p>Assuming this works properly for telling if...,"PHP, 'Haystack contains Needle?'",<php>
2,35064632,<p>I'm trying to understand substantial speed ...,Understanding json_agg performance in Postgres...,<performance><postgresql>
3,42515650,<p>I have a simple java program. A main thread...,java: why main thread waits for child thread t...,<java><multithreading>
4,8594144,<p>I want to stream live video to a browser wi...,Stream live video to browser (with low latency),<flash><video><stream><live>


In [54]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      40000 non-null  int64 
 1   Body    40000 non-null  object
 2   Title   40000 non-null  object
 3   Tags    40000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


In [55]:
#df_sample = df_sample.loc[:, ['Id', 'Title', 'Body', 'Tags']]
df_sample = df_sample.set_index('Id').reset_index(drop=True)
df_sample.head()

,Body,Title,Tags
0,<p>Pandas does not seem to find all elements i...,Pandas not finding elements in Columns,<python><pandas>
1,<p>Assuming this works properly for telling if...,"PHP, 'Haystack contains Needle?'",<php>
2,<p>I'm trying to understand substantial speed ...,Understanding json_agg performance in Postgres...,<performance><postgresql>
3,<p>I have a simple java program. A main thread...,java: why main thread waits for child thread t...,<java><multithreading>
4,<p>I want to stream live video to a browser wi...,Stream live video to browser (with low latency),<flash><video><stream><live>


## **3.Preprocessing**

In [56]:
df_sample.duplicated(subset=["Body", "Title"]).sum()

0

In [57]:
example_body = df_sample.Body.iloc[0]

In [58]:
example_body

'<p>Pandas does not seem to find all elements in a list:</p>\n\n<pre><code>df = pd.DataFrame({"rid": ["125264429", "a"], "id": [1, 2]})\n1 in df["id"]                # &lt;- expect True, get True\n"125264429" in df["rid"]     # &lt;- expect True, get False\ndf[df["rid"] == "125264429"] # &lt;- yields result\n</code></pre>\n\n<p>I am sure there is a perfectly reasonable explanation for this behaviour, but I can\'t seem to find it. It seems that the last two columns contradict each other.\nDoes it have to do something with the fact that the datatype of the "rid" column is object?</p>\n'

In [59]:
example_title = df_sample.Title.iloc[0]

In [60]:
example_title

'Pandas not finding elements in Columns'

In [61]:
#Merge Title and Body
df_sample['Question'] = df_sample['Title'] + ' ' + df_sample['Body']

In [62]:
df_sample.head()


,Body,Title,Tags,Question
0,<p>Pandas does not seem to find all elements i...,Pandas not finding elements in Columns,<python><pandas>,Pandas not finding elements in Columns <p>Pand...
1,<p>Assuming this works properly for telling if...,"PHP, 'Haystack contains Needle?'",<php>,"PHP, 'Haystack contains Needle?' <p>Assuming t..."
2,<p>I'm trying to understand substantial speed ...,Understanding json_agg performance in Postgres...,<performance><postgresql>,Understanding json_agg performance in Postgres...
3,<p>I have a simple java program. A main thread...,java: why main thread waits for child thread t...,<java><multithreading>,java: why main thread waits for child thread t...
4,<p>I want to stream live video to a browser wi...,Stream live video to browser (with low latency),<flash><video><stream><live>,Stream live video to browser (with low latency...


In [63]:
variables_reorganization = ['Question', 'Tags']
df_sample = df_sample[['Question', 'Tags']]

In [64]:
df_sample.head()

,Question,Tags
0,Pandas not finding elements in Columns <p>Pand...,<python><pandas>
1,"PHP, 'Haystack contains Needle?' <p>Assuming t...",<php>
2,Understanding json_agg performance in Postgres...,<performance><postgresql>
3,java: why main thread waits for child thread t...,<java><multithreading>
4,Stream live video to browser (with low latency...,<flash><video><stream><live>


In [65]:
df_sample.Question.iloc[0]

'Pandas not finding elements in Columns <p>Pandas does not seem to find all elements in a list:</p>\n\n<pre><code>df = pd.DataFrame({"rid": ["125264429", "a"], "id": [1, 2]})\n1 in df["id"]                # &lt;- expect True, get True\n"125264429" in df["rid"]     # &lt;- expect True, get False\ndf[df["rid"] == "125264429"] # &lt;- yields result\n</code></pre>\n\n<p>I am sure there is a perfectly reasonable explanation for this behaviour, but I can\'t seem to find it. It seems that the last two columns contradict each other.\nDoes it have to do something with the fact that the datatype of the "rid" column is object?</p>\n'

In [66]:
df_sample.Question.iloc[17593]

"PHP DOMElement::getElementsByTagName - Anyway to get just the immediate matching children? <p>is there a way to retrieve only the immediate children found by a call to DOMElement::getElementsByTagName? For example, I have an XML document that has a category element. That category element has sub category elements (which have the same structure), like:</p>\n\n<pre><code>&lt;category&gt;\n    &lt;id&gt;1&lt;/id&gt;\n    &lt;name&gt;Top Level Category Name&lt;/name&gt;\n    &lt;subCategory&gt;\n        &lt;id&gt;2&lt;/id&gt;\n        &lt;name&gt;Sub Category Name&lt;/name&gt;\n    &lt;/subCategory&gt;\n    ...\n&lt;/category&gt;\n</code></pre>\n\n<p>If I have a DOMElement representing the top level category, </p>\n\n<pre><code>$topLevelCategoryElement-&gt;getElementsByTagName('id');\n</code></pre>\n\n<p>will return a list with the nodes for all 'id' elements, where I want just the one from the top level. Any way to do this outside of using XPath?</p>\n"

### **HTML**

In [67]:
#HTML characters removing
def remove_html(body):
  html_regex = re.compile('<.*?>') #Compile regular expresions
  return re.sub(html_regex, ' ', str(body)) # Replace regex by ' '

In [68]:
df_sample['Question'] = df_sample['Question'].apply(remove_html)
df_sample.head()

,Question,Tags
0,Pandas not finding elements in Columns Pandas...,<python><pandas>
1,"PHP, 'Haystack contains Needle?' Assuming thi...",<php>
2,Understanding json_agg performance in Postgres...,<performance><postgresql>
3,java: why main thread waits for child thread t...,<java><multithreading>
4,Stream live video to browser (with low latency...,<flash><video><stream><live>


In [69]:
df_sample.Question.iloc[0]

'Pandas not finding elements in Columns  Pandas does not seem to find all elements in a list: \n\n  df = pd.DataFrame({"rid": ["125264429", "a"], "id": [1, 2]})\n1 in df["id"]                # &lt;- expect True, get True\n"125264429" in df["rid"]     # &lt;- expect True, get False\ndf[df["rid"] == "125264429"] # &lt;- yields result\n  \n\n I am sure there is a perfectly reasonable explanation for this behaviour, but I can\'t seem to find it. It seems that the last two columns contradict each other.\nDoes it have to do something with the fact that the datatype of the "rid" column is object? \n'

### **URL**

In [70]:
#URL removing
def remove_url(body):
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
  return re.sub(url_regex, ' ', str(body))  

In [71]:
df_sample['Question'] = df_sample['Question'].apply(remove_url)
df_sample.head()

,Question,Tags
0,Pandas not finding elements in Columns Pandas...,<python><pandas>
1,"PHP, 'Haystack contains Needle?' Assuming thi...",<php>
2,Understanding json_agg performance in Postgres...,<performance><postgresql>
3,java: why main thread waits for child thread t...,<java><multithreading>
4,Stream live video to browser (with low latency...,<flash><video><stream><live>


In [72]:
df_sample.Question.iloc[0]

'Pandas not finding elements in Columns  Pandas does not seem to find all elements in a list: \n\n  df = pd.DataFrame({"rid": ["125264429", "a"], "id": [1, 2]})\n1 in df["id"]                # &lt;- expect True, get True\n"125264429" in df["rid"]     # &lt;- expect True, get False\ndf[df["rid"] == "125264429"] # &lt;- yields result\n  \n\n I am sure there is a perfectly reasonable explanation for this behaviour, but I can\'t seem to find it. It seems that the last two columns contradict each other.\nDoes it have to do something with the fact that the datatype of the "rid" column is object? \n'

### **Punctuation**

In [73]:
#Punctuation removing
def remove_punc(body):
  clean_text = re.sub(r'[?|!|"|:|=|_|{|}|[|]|-|$|%|^|&|]',r' ',str(body))
  clean_text = re.sub(r'[.|,|)|(|\|/|-|~|`|>|<|*|$|@|;|→|#|]',r' ', clean_text)
  return clean_text

In [74]:
df_sample['Question'] = df_sample['Question'].apply(remove_punc)

In [75]:
df_sample.Question.iloc[0]

" Pandas not finding elements in Columns  Pandas does not seem to find all elements in a list  \n\n  df   pd DataFrame   rid     125264429    a     id    1  2   \n1 in df  id                     lt   expect True  get True\n 125264429  in df  rid          lt   expect True  get False\ndf df  rid       125264429      lt   yields result\n  \n\n I am sure there is a perfectly reasonable explanation for this behaviour  but I can't seem to find it  It seems that the last two columns contradict each other \nDoes it have to do something with the fact that the datatype of the  rid  column is object   \n "

### **Other characters**

In [76]:
#All characters removing except letters, numbers 
#Lower
def remove_other(body):
  text = str(body)
  clean_text = re.sub(r"[^a-zA-Z0-9#+-]", " ", text.lower())
  return clean_text

In [77]:
df_sample['Question'] = df_sample['Question'].apply(remove_other)
df_sample.head()

,Question,Tags
0,pandas not finding elements in columns panda...,<python><pandas>
1,php haystack contains needle assuming th...,<php>
2,understanding json agg performance in postgre...,<performance><postgresql>
3,java why main thread waits for child thread ...,<java><multithreading>
4,stream live video to browser with low latenc...,<flash><video><stream><live>


In [78]:
df_sample.Question.iloc[0]

' pandas not finding elements in columns  pandas does not seem to find all elements in a list      df   pd dataframe   rid     125264429    a     id    1  2    1 in df  id                     lt   expect true  get true  125264429  in df  rid          lt   expect true  get false df df  rid       125264429      lt   yields result      i am sure there is a perfectly reasonable explanation for this behaviour  but i can t seem to find it  it seems that the last two columns contradict each other  does it have to do something with the fact that the datatype of the  rid  column is object     '

### **Space**

In [79]:
#Space removing
def remove_space(body):
  return ' '.join(str(body).split()) 

In [80]:
df_sample['Question'] = df_sample['Question'].apply(remove_space)

In [81]:
df_sample.Question.iloc[0]

'pandas not finding elements in columns pandas does not seem to find all elements in a list df pd dataframe rid 125264429 a id 1 2 1 in df id lt expect true get true 125264429 in df rid lt expect true get false df df rid 125264429 lt yields result i am sure there is a perfectly reasonable explanation for this behaviour but i can t seem to find it it seems that the last two columns contradict each other does it have to do something with the fact that the datatype of the rid column is object'

In [82]:
df_sample.Question.iloc[17593]

'php domelement getelementsbytagname anyway to get just the immediate matching children is there a way to retrieve only the immediate children found by a call to domelement getelementsbytagname for example i have an xml document that has a category element that category element has sub category elements which have the same structure like lt category gt lt id gt 1 lt id gt lt name gt top level category name lt name gt lt subcategory gt lt id gt 2 lt id gt lt name gt sub category name lt name gt lt subcategory gt lt category gt if i have a domelement representing the top level category toplevelcategoryelement gt getelementsbytagname id will return a list with the nodes for all id elements where i want just the one from the top level any way to do this outside of using xpath'

In [36]:
#df_analysis['Title_Body_count'] = df_analysis['Title_Body'].apply(lambda text: len(text.split(" ")))

In [84]:
df_sample.head()

,Question,Tags
0,pandas not finding elements in columns pandas ...,<python><pandas>
1,php haystack contains needle assuming this wor...,<php>
2,understanding json agg performance in postgres...,<performance><postgresql>
3,java why main thread waits for child thread to...,<java><multithreading>
4,stream live video to browser with low latency ...,<flash><video><stream><live>


### **StopWords**

In [85]:
#Tokenization
#nltk.download("punkt")
#from nltk.tokenize import word_tokenize

In [86]:
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.stem.snowball import EnglishStemmer
lemma = WordNetLemmatizer().lemmatize
ps = PorterStemmer()
stemmer = EnglishStemmer()

In [87]:
set_alpha = set(['a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 's', 't', 'u', 'v', 'w', 'x', 'y' 'z'])

In [88]:
def remove_stopwords(text):
    text = str(text)
    text = " ".join(word for word in text.split() if word not in gensim.parsing.preprocessing.STOPWORDS and word not in stop_words and word not in set_alpha)
    return text

In [89]:
df_sample['Question'] = df_sample['Question'].apply(remove_stopwords)

In [90]:
df_sample.Question.iloc[0]

'pandas finding elements columns pandas elements list df pd dataframe rid 125264429 id 1 2 1 df id lt expect true true 125264429 df rid lt expect true false df df rid 125264429 lt yields result sure perfectly reasonable explanation behaviour columns contradict fact datatype rid column object'

In [91]:
df_sample.Question.iloc[17593]

'php domelement getelementsbytagname immediate matching children way retrieve immediate children domelement getelementsbytagname example xml document category element category element sub category elements structure like lt category gt lt id gt 1 lt id gt lt gt level category lt gt lt subcategory gt lt id gt 2 lt id gt lt gt sub category lt gt lt subcategory gt lt category gt domelement representing level category toplevelcategoryelement gt getelementsbytagname id return list nodes id elements want level way outside xpath'

In [92]:
df_sample.head()

,Question,Tags
0,pandas finding elements columns pandas element...,<python><pandas>
1,php haystack contains needle assuming works pr...,<php>
2,understanding json agg performance postgres 9 ...,<performance><postgresql>
3,java main thread waits child thread finish sim...,<java><multithreading>
4,stream live video browser low latency want str...,<flash><video><stream><live>


## **Tokenization**

### **Question**

In [93]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [94]:
def tokenize(document):
  tokens = [lemma(w) for w in document.split()] #if w.isalpha()
  return tokens

In [96]:
df_sample['Question_Tokens'] = df_sample['Question'].apply(tokenize)

In [97]:
df_sample.head()

,Question,Tags,Question_Tokens
0,pandas finding elements columns pandas element...,<python><pandas>,"[panda, finding, element, column, panda, eleme..."
1,php haystack contains needle assuming works pr...,<php>,"[php, haystack, contains, needle, assuming, wo..."
2,understanding json agg performance postgres 9 ...,<performance><postgresql>,"[understanding, json, agg, performance, postgr..."
3,java main thread waits child thread finish sim...,<java><multithreading>,"[java, main, thread, wait, child, thread, fini..."
4,stream live video browser low latency want str...,<flash><video><stream><live>,"[stream, live, video, browser, low, latency, w..."


In [100]:
print(df_sample.Question_Tokens.iloc[17593])

['php', 'domelement', 'getelementsbytagname', 'immediate', 'matching', 'child', 'way', 'retrieve', 'immediate', 'child', 'domelement', 'getelementsbytagname', 'example', 'xml', 'document', 'category', 'element', 'category', 'element', 'sub', 'category', 'element', 'structure', 'like', 'lt', 'category', 'gt', 'lt', 'id', 'gt', '1', 'lt', 'id', 'gt', 'lt', 'gt', 'level', 'category', 'lt', 'gt', 'lt', 'subcategory', 'gt', 'lt', 'id', 'gt', '2', 'lt', 'id', 'gt', 'lt', 'gt', 'sub', 'category', 'lt', 'gt', 'lt', 'subcategory', 'gt', 'lt', 'category', 'gt', 'domelement', 'representing', 'level', 'category', 'toplevelcategoryelement', 'gt', 'getelementsbytagname', 'id', 'return', 'list', 'node', 'id', 'element', 'want', 'level', 'way', 'outside', 'xpath']


## **Tags**

In [ ]:
from ast import literal_eval

In [ ]:
df_sample.Tags.iloc[0]

'<python><pandas>'

In [ ]:
# convert data to list
to_list = {'><': "','",'<':"['", '>':"']"}
for key, value in to_list.items():
  df_sample['Tags'] = df_sample['Tags'].str.replace(key, value)

In [ ]:
df_sample_tok = df_sample
df_sample_tok.head()

,Question,Tags,Question_Tokens
0,pandas finding elements columns pandas element...,"['python','pandas']","[panda, finding, element, column, panda, eleme..."
1,php haystack contains needle assuming works pr...,['php'],"[php, haystack, contains, needle, assuming, wo..."
2,understanding json agg performance postgres 9 ...,"['performance','postgresql']","[understanding, json, agg, performance, postgr..."
3,java main thread waits child thread finish sim...,"['java','multithreading']","[java, main, thread, wait, child, thread, fini..."
4,stream live video browser low latency want str...,"['flash','video','stream','live']","[stream, live, video, browser, low, latency, w..."


In [ ]:
def to_list(df,column):
  df[column] = df[column].apply(literal_eval)
  return df

In [ ]:
df_sample_list = to_list(df_sample_tok, 'Tags')

In [ ]:
reorganization = ['Question', 'Question_Tokens', 'Tags']
df_sample_list = df_sample_list[reorganization]
df_sample_list.head()


,Question,Question_Tokens,Tags
0,pandas finding elements columns pandas element...,"[panda, finding, element, column, panda, eleme...","[python, pandas]"
1,php haystack contains needle assuming works pr...,"[php, haystack, contains, needle, assuming, wo...",[php]
2,understanding json agg performance postgres 9 ...,"[understanding, json, agg, performance, postgr...","[performance, postgresql]"
3,java main thread waits child thread finish sim...,"[java, main, thread, wait, child, thread, fini...","[java, multithreading]"
4,stream live video browser low latency want str...,"[stream, live, video, browser, low, latency, w...","[flash, video, stream, live]"


In [ ]:
df_sample_list.to_csv('df_sample_list.csv')

In [ ]:
#df_mod = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/df_sample_list.csv')
#df_mod = df_mod.iloc[:, 1:]
#df_mod.head()

In [ ]:
#questions2 = questions1.apply(tokenize)
#questions2